In [1]:
# TODO: Remove this before submission
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

[nltk_data] Downloading package wordnet to /Users/aditya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

## Read Data

## Keep Reviews and Ratings

In [4]:
# Load the tab separated data file, and print the first 5 rows for confirmation
data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])
data.head()

/var/folders/d7/gcxbjppx70qfkdwy32kg04dh0000gn/T/ipykernel_5923/2961754961.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("./data/amazon_reviews_us_Jewelry_v1_00.tsv", sep="\t", usecols=["star_rating", "review_body"])


,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


Understanding Data

In [5]:
data.describe()

,star_rating,review_body
count,1767042,1766807
unique,11,1618522
top,5,Love it
freq,1041056,4288


In [7]:
data.star_rating.unique()

array([5, 1, 4, 3, 2, nan, '5', '1', '3', '4', '2', '2012-12-21'],
      dtype=object)

In [8]:
data.groupby(["star_rating"]).count()

,review_body
star_rating,
1,150441
2,97259
3,153660
4,259019
5,1040896
1,4566
2,3541
2012-12-21,0
3,5999


In [9]:
# Drop the outlier which is star_rating = "2012-12-21"
data = data[data.star_rating != "2012-12-21"]

In [10]:
# Remove nan valued rows
data = data[data.star_rating.notnull()]

In [11]:
data.describe()

,star_rating,review_body
count,1767041,1766807
unique,10,1618522
top,5,Love it
freq,1041056,4288


In [12]:
# Convert all star rating to integer
data['star_rating'] = data.star_rating.astype(int)

In [13]:
data.head()

,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [14]:
data = data[data.review_body.notnull()]
data.head()

,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...


In [15]:
# There are no empty reviews
(data.review_body.str.len() <= 0).sum()

0

Now we can continue with the process.

 ## We select 20000 reviews randomly from each rating class.



In [16]:
np.random.seed(17)
N_SAMPLES = 20000

In [17]:
reduced_data = data.groupby('star_rating', group_keys=False).apply(lambda x: x.sample(N_SAMPLES))

In [19]:
# TODO: Remove Test Block
# Checkout: Sampled Review, Ratings Data
# Save data to a file for having a look at the text content

reduced_data.head()
reduced_data.to_csv("./data/training.tsv", sep="\t", index=False, encoding="UTF-8")

,star_rating,review_body
1121877,1,"The necklace is nice but there is no clasp, ho..."
1081914,1,"I get that this is a small plastic box, and it..."
819005,1,Doesn't run true to size and sits up too high !!
1253749,1,"This is cute, but cheap. It's very stiff and ..."
1178474,1,They never got this item.<br />I was reinburse...


# Data Cleaning



In [42]:
avg_len_before_cleaning = reduced_data.review_body.str.len().mean()
f"Avg. length of reviews BEFORE CLEANING :: {avg_len_before_cleaning}"

'Avg. length of reviews BEFORE CLEANING :: 189.85384'

In [20]:
# TODO: Remove Test Block
# Test to lower case
reduced_data["review_body"].str.lower()

1121877    the necklace is nice but there is no clasp, ho...
1081914    i get that this is a small plastic box, and it...
819005      doesn't run true to size and sits up too high !!
1253749    this is cute, but cheap.  it's very stiff and ...
1178474    they never got this item.<br />i was reinburse...
                                 ...                        
1429112    exactly what the stars indicate.  a great piec...
275415                                 beautiful earrings!!!
817521     my gf loved it and wears it pretty much everyd...
454028     this is one of the prettiest rings i've seen. ...
1289078    these earrings are beautiful and add class to ...
Name: review_body, Length: 100000, dtype: object

In [21]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: pd.Series):
  return data.str.lower()

In [25]:
# TODO: Remove Test Block
import unicodedata

# Remove accented characters
print(unicodedata.normalize('NFKD', "Sómě Áccěntěd těxt. Some words such as résumé, café, prótest, divorcé, coördinate, exposé, latté.").encode('ascii', 'ignore').decode('utf-8', 'ignore'))

Some Accented text. Some words such as resume, cafe, protest, divorce, coordinate, expose, latte.


In [26]:
def remove_accented_characters(data: pd.Series):
  """Removes accented characters from the Series

  Args:
      data (pd.Series): Series of string

  Returns:
      _type_: pd.Series
  """
  import unicodedata

  return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))

In [27]:
# TODO: Remove Test Block
# Test HTML and URL replace RegEx
import re

# Removing HTML tags with and without closing tags
re.sub(r"<[a-zA-Z]+\s?/?>", " ", "this is a string <containing> some random XML tags <br> which might not be closed like this <br /> or this <br/>")

# Removing HTML encoded characters.
re.sub(r"&#\d+;", " ", "why are &#34; you doing this?")

# Removing URLs
re.sub(r"https?://([\w\-\.]+){2,}/[\w\-\.\-/=\+]+", " ", "Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\""The more I give to you, the more I have;<br />For both are infinite.\\""<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Garnet by night:<br />To stand right in Our Saviour's sight;<br />This is our health;<br />And all of our wealth. --- Gwen Newclear The Poet<br /><br />I thought I lost this ring:  found it.  http://poetgwennewclear - dot - sharepoint - dot - com")

'this is a string   some random XML tags   which might not be closed like this   or this  '

'why are   you doing this?'

"Ruby Garnet Ring<br />I saw this ring after I had already purchased 4 rings for my birthday.  I just could not resist buying this ring because of its color (and the price was so low I couldn't believe it!)<br />It arrived two days before my birthday in a small black velvet ring bag with a draw string.  It was so pretty and so perfect.  It looked like Ruby during the daylight.  But at night, under incandescent lighting, it looked garnet.  I have owned and worn many, many rings in the past 10 years; but this ring looked the best of all on my hand.  I have decided to name this ring:<br />Ruby Garnet for a Renewed America<br />And I wrote a little poem for meditation to go along with it:<br /><br />Ruby Garnet for a Renewed America<br />Ruby by day:<br />Giving freely each day<br />Is to give `the Juliet way':<br /><br />\\The more I give to you, the more I have;<br />For both are infinite.\\<br /><br />To give freely is right;<br />Standing favorably in Our Saviour's sight.<br /><br />Ga

In [28]:
def remove_html_and_url(data: pd.Series):
  """Function to remove
           1. HTML encodings
           2. HTML tags (both closed and open)
           3. URLs

  Args:
      data (pd.Series): A Pandas series of type string

  Returns:
      _type_: pd.Series
  """
  # Remove HTML encodings
  data.str.replace(r"&#\d+;", " ", regex=True)

  # Remove HTML tags (both open and closed)
  data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

  # Remove URLs
  data.str.replace(r"https?://([\w\-\.]+){2,}/[\w\-\.\-/=\+]+", " ", regex=True)

  return data

In [49]:
# TODO: Remove Test Block
# Test Non-alpha characters replace RegEX
import re

# Remove non-alphabetical characters
re.sub(r"[\^#]|[^a-zA-z0-9\s]", "", "33245 sure# if this % was #not! 55888392923 What do# you will** of it.? $$^!")

'33245 sure if this  was not 55888392923 What do you will of it '

In [29]:
# Remove non-alphabetical characters
def remove_non_alpa_characters(data: pd.Series):
  return data.str.replace(r"[\^#]|[^a-zA-z0-9\s]", "", regex=True)

In [30]:
# TODO: Remove Test Block
# Test remove extra space RegEX
import re
re.sub(r"^\s*|\s\s*", " ", "sameple of the    template, where nothing \t is defined.")

' sameple of the template, where nothing is defined.'

In [31]:
# Remove extra spaces
def remove_extra_spaces(data: pd.Series):
  return data.str.replace(r"^\s*|\s\s*", " ", regex=True)

In [32]:
# Install contractions package, if you don't have it
! pip install contractions

In [33]:
# TODO: Remove Test Block
# Test expanding of contractions
import contractions

sample = "I was disappointed with this purchase, they were extremely small, can't even wear them, they should specify the size, I would've never purchased them if I had known... Thanks"
expanded = " ".join([contractions.fix(word) for word in sample.split()])
expanded

'I was disappointed with this purchase, they were extremely small, cannot even wear them, they should specify the size, I would have never purchased them if I had known... Thanks'

In [44]:
# Expanding contractions
def fix_contractions(data: pd.Series):
  import contractions

  def contraction_fixer(txt: str):
    return " ".join([contractions.fix(word) for word in txt.split()])

  return data.apply(contraction_fixer)

In [45]:
# A dictionary containing the columns and a list of functions to perform on it in order
# TODO: Think about handling negation.
data_cleaning_pipeline = {
  "review_body": [
    to_lower, remove_accented_characters, remove_html_and_url, fix_contractions, remove_non_alpa_characters, remove_extra_spaces,
  ]
}

cleaned_data = data

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
  # Get the column to perform cleaning on
  temp_data = data[col]

  # Perform all the cleaning functions sequencially
  for func in pipeline:
    print(f"Starting: {func.__name__}")
    temp_data = func(temp_data)
    print(f"Ended: {func.__name__}")

  # Replace the old column with cleaned one.
  cleaned_data[col] = temp_data


In [46]:
avg_len_after_cleaning = cleaned_data.review_body.str.len().mean()
f"Avg. length of reviews after cleaning :: {avg_len_after_cleaning}"

'Avg. length of reviews after cleaning :: 171.91200906494032'

In [47]:
# TODO: Remove Test Block
# Checkpoint: Cleaned Data
cleaned_data.to_csv("./data/cleaned.tsv", sep="\t", index=False, encoding="UTF-8")

# Pre-processing

## remove the stop words 

In [ ]:
from nltk.corpus import stopwords



## perform lemmatization  

In [ ]:
from nltk.stem import WordNetLemmatizer


# TF-IDF Feature Extraction

# Perceptron

# SVM

# Logistic Regression

# Naive Bayes